In [ ]:
# instsall tar2tf module
! pip3 install ../ais

In [ ]:
# list available ais buckets
!ais create bucket imagenet-tar
!ais ls ais://

In [22]:
!ais start download "gs://lpr-gtc2020/oisubset-train-{0000..0003}.tar" ais://imagenet-tar

aaruNGayy
Run `ais show download aaruNGayy` to monitor the progress of downloading.


In [ ]:
# list contents of your bucket
# wait until download is finished.
!ais ls ais://imagenet-tar

In [ ]:
import os
from random import randrange
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from ais_tar2tf import Dataset
from ais_tar2tf.ops import Select, Decode, Convert, Resize, Rotate, Func, Rename

EPOCHS = 5
BATCH_SIZE = 20

# ADJUST Dataset PARAMETERS BELOW

BUCKET_NAME = "imagenet-tar"
PROXY_URL = "http://172.17.0.2:8080"

INPUT_SHAPE = (224, 224, 3)

In [ ]:
def record_parser(record):
    keys_to_features = {
        "json": tf.io.FixedLenFeature([], tf.string),
        "img": tf.io.FixedLenFeature([], tf.string),
    }
    parsed = tf.io.parse_single_example(record, keys_to_features)
    # TODO: adjust label value based on "json" field.
    return tf.image.decode_jpeg(parsed["img"]), tf.cast(randrange(10), tf.int32)

In [ ]:
# Create Dataset.
# tar records will be transformed according to:
# Rename(img="png;jpg;jpeg"), Decode("img"), Rotate("img"), Resize("img", (224, 224)) operations,
# meaning that png, jpg and jpeg will be renamed to img, then bytes under "img" in tar-record will be decoded as an image,
# Rotated by random angle and then Resized to (224, 224).
# Datapoints will be constructed from "img" and "json" tar records entries
conversions = [Rename(img="png;jpg;jpeg"), Decode("img"), Rotate("img"), Resize("img", (224, 224))]
selections = ["img", "json"]

# Initialization of an Dataset.
# It may take a moment as it starts ETL containers in the cluster.
# After finished working with Dataset, call dataset.stop()
dataset = Dataset(BUCKET_NAME, PROXY_URL, conversions, selections)

In [ ]:
# Remote conversions and selections execution is enabled by default.
# Datasets will be prepared from "oisubset-train-{0000..0003}.tar" files.
# Prefetches dataset, repeats it 100 times and splits into batches of size BATCH_SIZE
train_dataset = dataset.load("oisubset-train-{0000..00001}.tar", record_to_example=record_parser
).cache().repeat(100).batch(BATCH_SIZE)

test_dataset = dataset.load("oisubset-train-{0002..0003}.tar", record_to_example=record_parser
).cache().repeat(100).batch(BATCH_SIZE)

In [ ]:
# TRAINING PART BELOW
inputs = keras.Input(shape=(
    224,
    224,
    3,
), name="images")
x = layers.Flatten()(inputs)
x = layers.Dense(64, activation="relu", name="dense_1")(x)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=keras.optimizers.Adam(1e-4), loss=keras.losses.mean_squared_error, metrics=["acc"])
model.summary()

model.fit(train_dataset, epochs=EPOCHS, steps_per_epoch=BATCH_SIZE)
result = model.evaluate(test_dataset)
print(dict(zip(model.metrics_names, result)))


In [ ]:
# Stop created in the cluster ETL.
dataset.stop()